In [80]:
# Library imports
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Snap2Metis import metis_map
import pickle

# Prunning Youtube Network for Analysis

In [81]:
youtubeG = nx.read_edgelist("data/com-youtube.ungraph.txt", create_using = nx.Graph, nodetype = int)

FileNotFoundError: [Errno 2] No such file or directory: 'data/com-youtube.ungraph.txt'

The next cell will make the youtube network much smaller so we can 

In [ ]:

# Obtains list of degrees that have degree less than 3
degree_Thres = 6
remove = [node for node,degree in dict(youtubeG.degree()).items() if  degree < degree_Thres]

# Removes nodes with small degree and creates sparse matrix
youtubeG.remove_nodes_from(remove)
nx.write_edgelist(youtubeG,'youtube_sparse.txt', data=False)


# Cluster Analysis

Load in every graph (not YouTube yet). We'll look at some simple metrics to get some baseline understanding.

In [82]:
wikiG = nx.read_edgelist("input/wiki-Vote.txt", create_using = nx.Graph, nodetype = int)
collabG = nx.read_edgelist("input/ca-GrQc.txt", create_using = nx.Graph, nodetype = int)
p2pG = nx.read_edgelist("input/p2p-Gnutella08.txt", create_using = nx.Graph, nodetype = int)
fbG = nx.read_edgelist("input/facebook_combined.txt", create_using = nx.Graph, nodetype = int)
youtubeG = nx.read_edgelist("input/com-youtube.ungraph.txt", create_using = nx.Graph, nodetype = int)
    
pd.DataFrame({"Graph": ["Wikipedia", "Collaborations", "Gnutella", "Facebook", "Youtube"],
    "Node Count": [wikiG.number_of_nodes(), collabG.number_of_nodes(), p2pG.number_of_nodes(), fbG.number_of_nodes(), youtubeG.number_of_nodes()],
    "Edge Count": [wikiG.number_of_edges(), collabG.number_of_edges(), p2pG.number_of_edges(), fbG.number_of_edges(), youtubeG.number_of_edges()]})

,Graph,Node Count,Edge Count
0,Wikipedia,7115,100762
1,Collaborations,5242,14496
2,Gnutella,6301,20777
3,Facebook,4039,88234
4,Youtube,1134890,2987624


First we'll create our METIS mappings and save them as pickle files.

In [83]:
names = ["wiki-Vote", "ca-GrQc", "com-youtube.ungraph", "facebook_combined", "p2p-Gnutella08"]
for i in names:
    fname = "input/" + i + ".txt"
    obj_fname = "MetisAlgo/" + i + "_map.obj"
    metis_map(fname, obj_fname)

The graph is undirected.
7115 103689
The graph is undirected.
5242 28980
The graph is undirected.
1134890 2987624
The graph is undirected.
The graph is undirected.
6301 20777


The following functions will assist in loading our clusters and mappings.

In [90]:
#pulls the map from Snap2Metis, converts to list where index = Metis_NodeID and value = Snap_NodeID
def get_map(name):
    mmap = pickle.load(open("MetisAlgo/" + name + "_map.obj", "rb"))
    mlist = [ele for ele in mmap]
    return mmap, mlist

#get clustering for _community.txt files
def community_clusters(fname):
    #to-do
    return 0

#given (name and type) -> fname, load clustering and convert to list of communities (each community is its own list)
def get_clustering(name, type):

    #get fname
    if type == "mcl":
        suffix = ".metis.part.100" #not sure if I mixed up "mcl" and "metis" file suffixes
    elif type == "metis":
        suffix = ".metis.c1000.i2.0.b0.5" #this suffix as well ^^
    elif type == "community":
        fname = "output/" + name + "_community.txt"
        return community_clusters(fname)
    fname = "output/" + name + suffix

    clustering = np.loadtxt(fname, dtype=int) #load clustering
    mmap, mlist = get_map(name)
    clusters = [[] for i in range(len(np.unique(clustering)))] #group by cluster
    for i in range(len(clustering)):
        clusters[clustering[i]].append(mlist[i])
    return clusters

#hopefully print results, modularity only works for wikiG (so setting it to zero)
def print_results(G, name, type):
    clusters = get_clustering(name, type)
    results = []
    V = set(G.nodes) #does not change
    modularity = nx.algorithms.community.modularity(G, clusters) #does not change
    for i in range(100):
        C = set(clusters[i])
        C_bar = V - C
        ncut = nx.cut_size(G, C, C_bar) / nx.cut_size(G, C, V)
        conductance = nx.conductance(G, C)
        results.append({"Modularity": modularity, "n-cut": ncut, "Conductance": conductance})
    return pd.DataFrame(results)

### Wiki-Vote

In [86]:
print_results(wikiG, "wiki-Vote", "metis")

,Modularity,n-cut,Conductance
0,0.010801,0.864518,0.761366
1,0.010801,0.995062,0.990172
2,0.010801,0.824171,0.700928
3,0.010801,0.996769,0.993559
4,0.010801,0.997680,0.995370
...,...,...,...
95,0.010801,1.000000,1.000000
96,0.010801,1.000000,1.000000
97,0.010801,0.994186,0.988439
98,0.010801,0.995868,0.991770


### Collaborations

In [97]:
print_results(collabG, names[1], "metis") #this case does not work for some reason

NotAPartition: [[3466, 22074, 1407, 22483, 11464, 1383, 15154], [937, 15659, 23066, 25785, 17817, 22375, 16718, 8676], [5233, 11442, 15252], [8579, 19489, 17075, 25382, 9485, 18457, 22791, 21754, 1694, 3265, 88, 18778, 23712, 20345, 20307, 18233, 12797, 19442, 3632, 5083, 25998, 14841, 11828, 15227], [10310, 15931, 1356, 20562, 12627, 20000, 21469, 17240, 16230, 15187, 2142, 1879, 18366, 9579, 16314], [17038, 10268, 11402, 4868, 6160, 6023, 4371, 4442, 23723, 6303, 18764, 9491, 21660, 6810, 20414, 20062, 23620, 17696], [18720, 23721, 8868, 21491, 22530, 16766, 16620, 299, 16115, 11114, 10391, 6229, 24470, 2329, 22927, 1187, 5364], [19607, 1727, 46, 1520, 8701, 25250, 4634, 5938, 14078, 6878, 24169, 5415, 25599, 7596], [1854, 1738, 6427, 15961, 24110, 17443, 19678, 10637, 20261, 1279, 5144], [4583, 3927, 10871, 11223, 17382, 2455, 10910, 11939, 20537, 19956, 20560, 9149, 18091, 13620], [9572, 6575, 1375, 4642, 22699, 17991, 5529, 15372, 18217, 10338, 15681, 15259, 24141], [10841, 17439,

### Youtube

In [ ]:
print_results(youtubeG, names[2], "mcl")

### Facebook

In [93]:
print_results(fbG, "facebook_combined", "mcl")

,Modularity,n-cut,Conductance
0,0.022139,0.989519,0.979255
1,0.022139,0.988159,0.976596
2,0.022139,0.990840,0.981846
3,0.022139,0.988824,0.977895
4,0.022139,0.988095,0.976471
...,...,...,...
95,0.022139,0.991874,0.983879
96,0.022139,0.977131,0.955285
97,0.022139,0.979416,0.959662
98,0.022139,0.981105,0.962911


### Peer-To-Peer Filesharing

In [100]:
print_results(p2pG, names[4], "metis")

,Modularity,n-cut,Conductance
0,0.006187,0.869565,0.769231
1,0.006187,1.000000,1.000000
2,0.006187,1.000000,1.000000
3,0.006187,1.000000,1.000000
4,0.006187,0.981678,0.964016
...,...,...,...
95,0.006187,1.000000,1.000000
96,0.006187,1.000000,1.000000
97,0.006187,1.000000,1.000000
98,0.006187,1.000000,1.000000
